<a href="https://colab.research.google.com/github/syusuke9999/FashionMNIST_VGG16/blob/main/FashionMNIST_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import keras
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.applications import VGG16
import numpy as np
import logging
logging.getLogger('tensorflow').disabled = True

ImportError: ignored

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt

**パラメーターの設定**

In [ ]:
batch_size = 128
num_class = 10
epochs = 100
image_size = 32

In [ ]:
#  keras.datasets.fashion_mnistからデータを読み込む。
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

In [ ]:
#  それぞれの分類をリストに保存する
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

####  VGG16のトレーニングモデルは最低限32×32のサイズとRGBカラーチャネルを持ったデータでないと受け付けないため、
#### リスト内包表記とopenCVの関数を使用して、画像サイズを拡大し、カラーチャネルをRGBの3つに増やす。
####  x_trainからimgを1つずつ取り出し、画像サイズの変換とRGBデータへの変換を同時に行い、numpy.ndarray形式に変換して、x_trainに代入する

In [ ]:
x_train = np.array(
    [cv2.cvtColor(cv2.resize(img, (image_size, image_size)),
                  cv2.COLOR_GRAY2RGB) for img in x_train])

###  x_testについても同様の操作を行う

In [ ]:
x_test = np.array(
    [cv2.cvtColor(cv2.resize(img, (image_size, image_size)),
                  cv2.COLOR_GRAY2RGB) for img in x_test])

In [ ]:
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.title("Label: " + str(i))
    plt.imshow(x_train[i].reshape(image_size, image_size, 3), cmap=None)

In [ ]:
# 前処理として画像データを0～1の間の数値に正規化する
x_train, x_test = x_train / 255.0 , x_test / 255.0

In [ ]:
x_test.shape

In [ ]:
# 最適化手法の指定
opt = Adam()

In [ ]:
# 事前学習に'imagenet'で学習したモデルを使用（weights='imagenet'）
# 今回は10クラスに分類するため、最後尾（トップ）の全結合層は使用しない（include_top=False）
# 画像サイズとして最低32×32、カラーチャネルとしてRGBの3の次元が必要なため、
# 事前に加工した形状の通りに入力の形状を指定（input_shape=(image_size, image_size, 3)）
VGG16TrainedModel = VGG16(weights='imagenet', include_top=False,
              input_shape=(32, 32, 3))

### VGG16の事前学習済みモデルの概要

In [ ]:
VGG16TrainedModel.summary()

### 全結合層の構築

In [ ]:
top_model = Sequential()
top_model.add(Flatten(input_shape=VGG16TrainedModel.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(num_class, activation='softmax'))

### VGG16と全結合層の統合

In [ ]:
# 事前学習済みのVGG-16畳み込みニューラルネットワークと、上記作成したレイヤーを結合する
Customized_VGG_Model = keras.Model(inputs=VGG16TrainedModel.input, outputs=top_model(VGG16TrainedModel.output))

In [ ]:
# 事前学習済みのVGG-16畳み込みニューラルネットワークの13番目の層までを学習対象にする
for layer in Customized_VGG_Model.layers[:16]:
    layer.trainable = False

In [ ]:
### 今回の転移学習に使用するモデルの概要

In [ ]:
Customized_VGG_Model..summary()

In [ ]:
Customized_VGG_Model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
Customized_VGG_Model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

In [ ]:
Customized_VGG_Model.evaluate(x_test, y_test)

In [ ]:
plt.imshow(x_test[99:100].reshape(image_size, image_size, 3), cmap=None)

In [ ]:
class_names[Customized_VGG_Model.predict(x_test[99:100]).argmax()]